In [1]:
%store -z

In [2]:
import pandas as pd
import numpy as np
import missingno as msno 
import seaborn as sns
import matplotlib.pyplot as plt 

#import warnings
#warnings.filterwarnings('ignore')



In [3]:
# client data
df_clients =pd.read_csv('data/client_train.csv')

# invoice score
df_invoice =pd.read_csv('data/invoice_train.csv')

/var/folders/xy/6r8m_9zx6hx2xhzgg9rz5mhc0000gn/T/ipykernel_87034/3352888719.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_invoice =pd.read_csv('data/invoice_train.csv')


In [4]:
df_clients.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [21]:
df_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   disrict        135493 non-null  int64  
 1   client_id      135493 non-null  object 
 2   client_catg    135493 non-null  int64  
 3   region         135493 non-null  int64  
 4   creation_date  135493 non-null  object 
 5   target         135493 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 6.2+ MB


In [5]:
df_invoice.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC


In [6]:
df_invoice[['client_id', 'invoice_date', 'old_index', 'new_index']].query('client_id == "train_Client_0"').sort_values(by=['invoice_date'])

,client_id,invoice_date,old_index,new_index
22,train_Client_0,2005-10-17,3685,3809
23,train_Client_0,2006-02-24,3809,3950
24,train_Client_0,2006-06-23,3950,4112
25,train_Client_0,2006-10-18,4112,4271
28,train_Client_0,2007-02-26,4271,4453
26,train_Client_0,2007-06-27,4453,4693
27,train_Client_0,2007-10-25,4693,4969
34,train_Client_0,2008-01-04,4969,5246
29,train_Client_0,2008-07-28,5246,5417
10,train_Client_0,2008-11-25,5417,5591


## Column descriptions:

### Client Data

* Client_id: Unique id for client
* District: District where the client is
* Client_catg: Category client belongs to
* Region: Area where the client is
* Creation_date: Date client joined
* Target: fraud:1 , not fraud: 0

### Invoice Data

* Client_id: Unique id for the client
* Invoice_date: Date of the invoice
* Tarif_type: Type of tax
* Counter_number:
* Counter_statue: takes up to 5 values such as working fine, not working, on hold statue, ect
* Counter_code:
* Reading_remarque: notes that the STEG agent takes during his visit to the client (e.g: If the counter shows something wrong, the agent gives a bad score)
* Counter_coefficient: An additional coefficient to be added when standard consumption is exceeded
* Consommation_level_1: Consumption_level_1
* Consommation_level_2: Consumption_level_2
* Consommation_level_3: Consumption_level_3
* Consommation_level_4: Consumption_level_4
* Old_index: Old index
* New_index: New index
* Months_number: Month number
* Counter_type: Type of counter

 the consumption level refers to the threshold of consumption to which a certain price is attributed

In [ ]:
#Y = coffee_quality["quality_score"]

# Data cleaningin and feature engineering

In [ ]:
df_clients.info()

In [ ]:
df_invoice.info()

In [ ]:
df_invoice.isnull().sum()

In [ ]:
df_invoice.describe().T

In [7]:
df_joined =  pd.merge(df_clients, df_invoice, on="client_id", how="left")
#df_clients.join(df_invoice, on='client_id', how='left')

In [8]:
df_joined.head(10)

,disrict,client_id,client_catg,region,creation_date,target,invoice_date,tarif_type,counter_number,counter_statue,...,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,60,train_Client_0,11,101,31/12/1994,0.0,2014-03-24,11,1335667,0,...,8,1,82,0,0,0,14302,14384,4,ELEC
1,60,train_Client_0,11,101,31/12/1994,0.0,2013-03-29,11,1335667,0,...,6,1,1200,184,0,0,12294,13678,4,ELEC
2,60,train_Client_0,11,101,31/12/1994,0.0,2015-03-23,11,1335667,0,...,8,1,123,0,0,0,14624,14747,4,ELEC
3,60,train_Client_0,11,101,31/12/1994,0.0,2015-07-13,11,1335667,0,...,8,1,102,0,0,0,14747,14849,4,ELEC
4,60,train_Client_0,11,101,31/12/1994,0.0,2016-11-17,11,1335667,0,...,9,1,572,0,0,0,15066,15638,12,ELEC
5,60,train_Client_0,11,101,31/12/1994,0.0,2017-07-17,11,1335667,0,...,9,1,314,0,0,0,15638,15952,8,ELEC
6,60,train_Client_0,11,101,31/12/1994,0.0,2018-12-07,11,1335667,0,...,9,1,541,0,0,0,15952,16493,12,ELEC
7,60,train_Client_0,11,101,31/12/1994,0.0,2019-03-19,11,1335667,0,...,9,1,585,0,0,0,16493,17078,8,ELEC
8,60,train_Client_0,11,101,31/12/1994,0.0,2011-07-22,11,1335667,0,...,9,1,1200,186,0,0,7770,9156,4,ELEC
9,60,train_Client_0,11,101,31/12/1994,0.0,2011-11-22,11,1335667,0,...,6,1,1082,0,0,0,9156,10238,4,ELEC


In [9]:
print(df_joined.client_id.nunique())
df_joined.counter_number.nunique()

135493


201893

In [ ]:
df_joined.nunique()

In [ ]:
df_joined.counter_type.unique()

In [ ]:
df_joined.client_catg.unique()

In [ ]:
df_joined.tarif_type.unique()

In [ ]:
df_joined.groupby('counter_statue').count()

In [ ]:
df_joined.groupby('counter_code').count()

In [ ]:
df_joined.counter_code.unique()

In [ ]:
df_joined.reading_remarque.unique()

In [ ]:
df_joined.groupby('reading_remarque').count()

In [ ]:
df_joined.counter_coefficient.unique()

In [ ]:
df_joined.old_index.unique()

In [10]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476749 entries, 0 to 4476748
Data columns (total 21 columns):
 #   Column                Dtype  
---  ------                -----  
 0   disrict               int64  
 1   client_id             object 
 2   client_catg           int64  
 3   region                int64  
 4   creation_date         object 
 5   target                float64
 6   invoice_date          object 
 7   tarif_type            int64  
 8   counter_number        int64  
 9   counter_statue        object 
 10  counter_code          int64  
 11  reading_remarque      int64  
 12  counter_coefficient   int64  
 13  consommation_level_1  int64  
 14  consommation_level_2  int64  
 15  consommation_level_3  int64  
 16  consommation_level_4  int64  
 17  old_index             int64  
 18  new_index             int64  
 19  months_number         int64  
 20  counter_type          object 
dtypes: float64(1), int64(15), object(5)
memory usage: 717.3+ MB


## To Dos

* remove client_id column
* turn into dummies: client_catg, tarif_type
* change target to 0,1 - DONE
* turn creation_date, invoice_date  into datetime or something else - DONE
* feature engineering: number of counters per client. then drop client_id, counter_number
* counter_statue: turn strings 0-5 into int, check percentage of values not 0-5, check for pattern, remove - DONE
* counter_code: either dummies or find info on steg site or drop?
* reading_remarque: clean? turn into dummies
* counter_coefficient: try both?: treat numerically and as dummies?
* rescale consommation_level1 ... _4
* drop index old and new
* rescale months_number
* turn into dummy counter_type

In [11]:
# check how many duplicated rows exist in the data frame
df_joined.duplicated().value_counts()

# there were 11 duplicatge rows, drop duplicates
df_joined.drop_duplicates(inplace=True)

In [12]:
df_joined['counter_statue'].unique()

array([0, 1, 5, 4, 3, 2, 769, '0', '5', '1', '4', 'A', 618, 269375, 46,
       420], dtype=object)

In [13]:
# counter_statue: turn strings 0-5 into int, check percentage of values not 0-5, check for pattern, remove

df_joined['counter_statue'] = df_joined['counter_statue'].map({
    '0': 0,
    0: 0, 
    1: 1,
    2 : 2,
    3: 3,
    4: 4,  
    5: 5,
    '5': 5,
    '1': 1,
    '4': 4, 
    'A': np.nan,
    618: np.nan, 
    269375: np.nan,
    46: np.nan, 
    420: np.nan,
    769: np.nan, 
    })
df_joined['counter_statue'].nunique()

6

In [14]:
df_joined['counter_statue'].unique()

array([ 0.,  1.,  5.,  4.,  3.,  2., nan])

In [ ]:
# check for missing distribution

#msno.matrix(df_joined)

In [15]:
# calculate fraction of data we would lose : 0.001%

print(f"numbers of rows : {df_joined.shape[0]}")
print(f"missing values in counter statue : {round(df_joined.counter_statue.isna().sum()/df_joined.shape[0]*100,4)} %")

# copy df
df_processed = df_joined.copy()
# drop NaN
df_processed.dropna(inplace=True, axis=0)


numbers of rows : 4476738
missing values in counter statue : 0.001 %


In [16]:
df_joined.isna().sum()

disrict                  0
client_id                0
client_catg              0
region                   0
creation_date            0
target                   0
invoice_date             0
tarif_type               0
counter_number           0
counter_statue          47
counter_code             0
reading_remarque         0
counter_coefficient      0
consommation_level_1     0
consommation_level_2     0
consommation_level_3     0
consommation_level_4     0
old_index                0
new_index                0
months_number            0
counter_type             0
dtype: int64

In [17]:
df_processed.isna().sum()

disrict                 0
client_id               0
client_catg             0
region                  0
creation_date           0
target                  0
invoice_date            0
tarif_type              0
counter_number          0
counter_statue          0
counter_code            0
reading_remarque        0
counter_coefficient     0
consommation_level_1    0
consommation_level_2    0
consommation_level_3    0
consommation_level_4    0
old_index               0
new_index               0
months_number           0
counter_type            0
dtype: int64

In [20]:
df_processed['target'].unique()

array([0., 1.])

In [22]:
# change target from float to int (0,1)
df_processed.target = df_processed.target.astype(int)
df_processed.target.unique()

array([0, 1])

In [23]:
df_processed.head()

,disrict,client_id,client_catg,region,creation_date,target,invoice_date,tarif_type,counter_number,counter_statue,...,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,60,train_Client_0,11,101,31/12/1994,0,2014-03-24,11,1335667,0.0,...,8,1,82,0,0,0,14302,14384,4,ELEC
1,60,train_Client_0,11,101,31/12/1994,0,2013-03-29,11,1335667,0.0,...,6,1,1200,184,0,0,12294,13678,4,ELEC
2,60,train_Client_0,11,101,31/12/1994,0,2015-03-23,11,1335667,0.0,...,8,1,123,0,0,0,14624,14747,4,ELEC
3,60,train_Client_0,11,101,31/12/1994,0,2015-07-13,11,1335667,0.0,...,8,1,102,0,0,0,14747,14849,4,ELEC
4,60,train_Client_0,11,101,31/12/1994,0,2016-11-17,11,1335667,0.0,...,9,1,572,0,0,0,15066,15638,12,ELEC


In [24]:
# turn columns invoice date and creation date to datetime

df_processed['invoice_date'] = pd.to_datetime(df_processed['invoice_date'], format='%Y-%m-%d')
df_processed['creation_date'] = pd.to_datetime(df_processed['creation_date'], format='%d/%m/%Y')

## Establishing Baseline Model

In [25]:
df_processed.columns

Index(['disrict', 'client_id', 'client_catg', 'region', 'creation_date',
       'target', 'invoice_date', 'tarif_type', 'counter_number',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number', 'counter_type'],
      dtype='object')

In [ ]:
# heatmap
fig = plt.figure(figsize=(20,10))
sns.heatmap(df_processed[['disrict', 'client_catg', 'region', 'creation_date',
       'target', 'invoice_date', 'tarif_type', 'counter_number',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number']].corr(), annot=True)



In [ ]:
# the pairplot took a lot of time so we did not see it so far and left it for now

# pairplot

# fig = plt.figure(figsize=(20,10))
# sns.pairplot(df_processed[['disrict', 'client_catg', 'region', 'creation_date', 'invoice_date', 'tarif_type', 'counter_number',
#        'counter_statue', 'counter_code', 'reading_remarque',
#        'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
#        'consommation_level_3', 'consommation_level_4', 'months_number', 'target']], hue='target')

In [ ]:

# fig = plt.figure(figsize=(20,10))
# sns.histplot(data= df_processed, x = 'reading_remarque', hue='target', stat='percent');

In [ ]:
# # since our target has the highest correlation with the client category, we will take a closer look at that: 
# df_processed.client_catg.value_counts()

In [ ]:
# # plot distribution of target for each client category 

# fig = plt.figure(figsize=(20,10))
# sns.countplot(data= df_processed, x = 'client_catg', hue='target');

In [ ]:
# # plot distribution of target for each client category individually, descending by clients in category:

# fig = plt.figure(figsize=(20,10))
# sns.histplot(data= df_processed.query('client_catg == 11'), x = 'client_catg', hue='target', stat='percent');

In [ ]:
# fig = plt.figure(figsize=(20,10))
# sns.histplot(data= df_processed.query('client_catg == 51'), x = 'client_catg', hue='target', stat='percent')

In [ ]:
# fig = plt.figure(figsize=(20,10))
# sns.histplot(data= df_processed.query('client_catg == 12'), x = 'client_catg', hue='target', stat='percent')

In [ ]:
# the highest amount of fraud happens by clients assigned to category 51, 
# so our very fist guess for a hypothesis and baseline mode could be:
# client category is the best predictor for fraud
# however, due tue the unbalanced data, this would be a very poor model.

## Next Steps
* run baseline model and print the different scores for it
* Drop old and new index
* Train-Test-Split
* Define Target y, and Features X
* Feature Engineering
* Dummy Creating
* Rescaling based on train, apply to test
* dropping other unused columns
* export X-test, X_train, y_test and y_train to a new notebook for modeling (find library to help us with that)
* individual modeling

## Feature Engineering

In [26]:
# month and year from date columns
df_processed['creation_month'] = df_processed['creation_date'].dt.month.astype(int)
df_processed['creation_year'] = df_processed['creation_date'].dt.year.astype(int)
df_processed['invoice_month'] = df_processed['invoice_date'].dt.month.astype(int)
df_processed['invoice_year'] = df_processed['invoice_date'].dt.year.astype(int)

In [27]:
# drop creation_date and invoice_date
df_processed.drop(['creation_date','invoice_date'] , inplace=True,axis=1)
df_processed.head()

,disrict,client_id,client_catg,region,target,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,...,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,creation_month,creation_year,invoice_month,invoice_year
0,60,train_Client_0,11,101,0,11,1335667,0.0,203,8,...,0,0,14302,14384,4,ELEC,12,1994,3,2014
1,60,train_Client_0,11,101,0,11,1335667,0.0,203,6,...,0,0,12294,13678,4,ELEC,12,1994,3,2013
2,60,train_Client_0,11,101,0,11,1335667,0.0,203,8,...,0,0,14624,14747,4,ELEC,12,1994,3,2015
3,60,train_Client_0,11,101,0,11,1335667,0.0,207,8,...,0,0,14747,14849,4,ELEC,12,1994,7,2015
4,60,train_Client_0,11,101,0,11,1335667,0.0,207,9,...,0,0,15066,15638,12,ELEC,12,1994,11,2016


In [28]:
# new feature index_change
df_processed['index_change']= df_processed['new_index'] - df_processed['old_index']

In [ ]:
#fig = plt.figure(figsize=(20,10))
#sns.histplot(data=df_processed, x='index_change' ,hue='target')

In [29]:
df_processed['index_change'].corr(df_processed['target']) #  correlation is very low 

0.009448350209865454

In [30]:
df_processed[['index_change']].describe()

,index_change
count,4.476691e+06
mean,5.825556e+02
std,4.608610e+03
min,-9.990960e+05
25%,7.900000e+01
50%,3.090000e+02
75%,6.560000e+02
max,6.332280e+05


In [38]:
quantile_idx = df_processed[['index_change']].quantile(q=0.15)[0]  # [0] give us the exactly value we need all our data below that 
quantile_idx

22.0

In [36]:
df_processed.query('index_change <= @quantile_idx')['target'].value_counts()

target
0    626366
1     50894
Name: count, dtype: int64

In [39]:
df_processed.drop(['old_index','new_index'] , inplace=True,axis=1)


In [40]:
df_processed

,disrict,client_id,client_catg,region,target,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,...,consommation_level_2,consommation_level_3,consommation_level_4,months_number,counter_type,creation_month,creation_year,invoice_month,invoice_year,index_change
0,60,train_Client_0,11,101,0,11,1335667,0.0,203,8,...,0,0,0,4,ELEC,12,1994,3,2014,82
1,60,train_Client_0,11,101,0,11,1335667,0.0,203,6,...,184,0,0,4,ELEC,12,1994,3,2013,1384
2,60,train_Client_0,11,101,0,11,1335667,0.0,203,8,...,0,0,0,4,ELEC,12,1994,3,2015,123
3,60,train_Client_0,11,101,0,11,1335667,0.0,207,8,...,0,0,0,4,ELEC,12,1994,7,2015,102
4,60,train_Client_0,11,101,0,11,1335667,0.0,207,9,...,0,0,0,12,ELEC,12,1994,11,2016,572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476744,60,train_Client_99998,11,101,0,10,1253571,0.0,202,9,...,135,0,0,8,ELEC,12,1993,8,2005,535
4476745,60,train_Client_99998,11,101,0,10,1253571,0.0,202,6,...,6,0,0,4,ELEC,12,1993,12,2005,206
4476746,60,train_Client_99999,11,101,0,11,560948,0.0,203,6,...,0,0,0,4,ELEC,2,1986,9,1996,259
4476747,60,train_Client_99999,11,101,0,11,560948,0.0,203,6,...,0,0,0,4,ELEC,2,1986,5,1996,603


In [41]:
# counter per client feature
quantity_counter = df_processed.groupby('client_id')['counter_number'].count().reset_index()
quantity_counter.rename(columns={'counter_number':'quantity_counters'}, inplace=True)
quantity_counter.head()

,client_id,quantity_counters
0,train_Client_0,35
1,train_Client_1,37
2,train_Client_10,18
3,train_Client_100,20
4,train_Client_1000,14


In [42]:
df_processed = pd.merge(df_processed, quantity_counter, on="client_id", how="left")
df_processed.head()

,disrict,client_id,client_catg,region,target,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,...,consommation_level_3,consommation_level_4,months_number,counter_type,creation_month,creation_year,invoice_month,invoice_year,index_change,quantity_counters
0,60,train_Client_0,11,101,0,11,1335667,0.0,203,8,...,0,0,4,ELEC,12,1994,3,2014,82,35
1,60,train_Client_0,11,101,0,11,1335667,0.0,203,6,...,0,0,4,ELEC,12,1994,3,2013,1384,35
2,60,train_Client_0,11,101,0,11,1335667,0.0,203,8,...,0,0,4,ELEC,12,1994,3,2015,123,35
3,60,train_Client_0,11,101,0,11,1335667,0.0,207,8,...,0,0,4,ELEC,12,1994,7,2015,102,35
4,60,train_Client_0,11,101,0,11,1335667,0.0,207,9,...,0,0,12,ELEC,12,1994,11,2016,572,35


In [43]:
df_processed['quantity_counters'].corr(df_processed['target']) #  correlation is very low 

0.09323153614744494

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.histplot(df_processed, x='quantity_counters', hue='target')

In [44]:
# drop client_id and counter_number
df_processed.drop(['client_id','counter_number'] , inplace=True,axis=1)
df_processed.head()

,disrict,client_catg,region,target,tarif_type,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,...,consommation_level_3,consommation_level_4,months_number,counter_type,creation_month,creation_year,invoice_month,invoice_year,index_change,quantity_counters
0,60,11,101,0,11,0.0,203,8,1,82,...,0,0,4,ELEC,12,1994,3,2014,82,35
1,60,11,101,0,11,0.0,203,6,1,1200,...,0,0,4,ELEC,12,1994,3,2013,1384,35
2,60,11,101,0,11,0.0,203,8,1,123,...,0,0,4,ELEC,12,1994,3,2015,123,35
3,60,11,101,0,11,0.0,207,8,1,102,...,0,0,4,ELEC,12,1994,7,2015,102,35
4,60,11,101,0,11,0.0,207,9,1,572,...,0,0,12,ELEC,12,1994,11,2016,572,35


In [45]:
# rename district column
df_processed.rename(columns={'disrict':'district'}, inplace=True)

## Heatmap with the new features

In [46]:
df_processed.columns

Index(['district', 'client_catg', 'region', 'target', 'tarif_type',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'months_number',
       'counter_type', 'creation_month', 'creation_year', 'invoice_month',
       'invoice_year', 'index_change', 'quantity_counters'],
      dtype='object')

In [ ]:
# # heatmap
# fig = plt.figure(figsize=(20,10))
# sns.heatmap(df_processed[['district', 'client_catg', 'region', 'target', 'tarif_type',
#        'counter_statue', 'counter_code', 'reading_remarque',
#        'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
#        'consommation_level_3', 'consommation_level_4', 'months_number',
#        'creation_month', 'creation_year', 'invoice_month',
#        'invoice_year', 'index_change', 'quantity_counters']].corr(), annot=True)

## Turning Categorical Features into Dummies

In [47]:
df_processed.columns.to_list()

['district',
 'client_catg',
 'region',
 'target',
 'tarif_type',
 'counter_statue',
 'counter_code',
 'reading_remarque',
 'counter_coefficient',
 'consommation_level_1',
 'consommation_level_2',
 'consommation_level_3',
 'consommation_level_4',
 'months_number',
 'counter_type',
 'creation_month',
 'creation_year',
 'invoice_month',
 'invoice_year',
 'index_change',
 'quantity_counters']

In [48]:
cat_features = ['district', 'client_catg', 'region', 'tarif_type', 'counter_statue', 'counter_code',
       'reading_remarque', 'counter_coefficient', 'counter_type']

In [49]:
num_features = [x for x in df_processed.columns.to_list() if x not in cat_features]
num_features

['target',
 'consommation_level_1',
 'consommation_level_2',
 'consommation_level_3',
 'consommation_level_4',
 'months_number',
 'creation_month',
 'creation_year',
 'invoice_month',
 'invoice_year',
 'index_change',
 'quantity_counters']

In [50]:
# remove target
num_features.remove("target")
num_features

['consommation_level_1',
 'consommation_level_2',
 'consommation_level_3',
 'consommation_level_4',
 'months_number',
 'creation_month',
 'creation_year',
 'invoice_month',
 'invoice_year',
 'index_change',
 'quantity_counters']

In [51]:
#make dummy variables from categorical features, dtype int
dummies = pd.get_dummies(df_processed[cat_features], columns=cat_features, drop_first=True, dtype=int)
dummies

,district_62,district_63,district_69,client_catg_12,client_catg_51,region_103,region_104,region_105,region_106,region_107,...,counter_coefficient_4,counter_coefficient_5,counter_coefficient_10,counter_coefficient_11,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_type_GAZ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476686,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4476687,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4476688,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4476689,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#%store -z

In [52]:
# add dummy variables to dataframe with dropped categorical columns
df_processed_dmy =  pd.concat([df_processed.drop(cat_features,axis=1), dummies],axis=1)
df_processed_dmy.head()

,target,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,months_number,creation_month,creation_year,invoice_month,invoice_year,...,counter_coefficient_4,counter_coefficient_5,counter_coefficient_10,counter_coefficient_11,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_type_GAZ
0,0,82,0,0,0,4,12,1994,3,2014,...,0,0,0,0,0,0,0,0,0,0
1,0,1200,184,0,0,4,12,1994,3,2013,...,0,0,0,0,0,0,0,0,0,0
2,0,123,0,0,0,4,12,1994,3,2015,...,0,0,0,0,0,0,0,0,0,0
3,0,102,0,0,0,4,12,1994,7,2015,...,0,0,0,0,0,0,0,0,0,0
4,0,572,0,0,0,12,12,1994,11,2016,...,0,0,0,0,0,0,0,0,0,0


In [56]:
df_processed_dmy.shape

(4476691, 117)

In [57]:
# store the preprocessed dataframe
df_processed_dmy.to_csv('data/df_processed_dmy.csv', index=False)

# # add to new notebooks:
# # load the preprocessed dataframe
# %store -r df_processed_dmy

In [ ]:
#%store -z

## Turning Categorical Features into Dummys for Tree-Models (no drop-first)

In [53]:
#make dummy variables from categorical features, dtype int, without dropping the first dummy columns
dummies_tree = pd.get_dummies(df_processed[cat_features], columns=cat_features, drop_first=False, dtype=int)
dummies_tree

,district_60,district_62,district_63,district_69,client_catg_11,client_catg_12,client_catg_51,region_101,region_103,region_104,...,counter_coefficient_5,counter_coefficient_10,counter_coefficient_11,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_type_ELEC,counter_type_GAZ
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476686,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4476687,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4476688,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4476689,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [54]:
# add dummy variables to dataframe with dropped categorical columns
df_processed_dmy_tree =  pd.concat([df_processed.drop(cat_features,axis=1), dummies_tree],axis=1)
df_processed_dmy_tree.head()

,target,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,months_number,creation_month,creation_year,invoice_month,invoice_year,...,counter_coefficient_5,counter_coefficient_10,counter_coefficient_11,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_type_ELEC,counter_type_GAZ
0,0,82,0,0,0,4,12,1994,3,2014,...,0,0,0,0,0,0,0,0,1,0
1,0,1200,184,0,0,4,12,1994,3,2013,...,0,0,0,0,0,0,0,0,1,0
2,0,123,0,0,0,4,12,1994,3,2015,...,0,0,0,0,0,0,0,0,1,0
3,0,102,0,0,0,4,12,1994,7,2015,...,0,0,0,0,0,0,0,0,1,0
4,0,572,0,0,0,12,12,1994,11,2016,...,0,0,0,0,0,0,0,0,1,0


In [58]:
# store the preprocessed dataframe for tree models
df_processed_dmy_tree.to_csv('data/df_processed_dmy_tree.csv', index=False)

## To Dos

* counter_code: find info on steg site or drop?
* reading_remarque: clean? 
* counter_coefficient: try both?: treat numerically and as dummies?
* rescale consommation_level1 ... _4
* rescale months_number

## Done
* rename 'disrict' column 'district'
* turn creation_date, invoice_date  into datetime
* change target to 0,1
* counter_statue: turn strings 0-5 into int, check percentage of values not 0-5, check for pattern, remove
* feature engineering: index_change = new_idex - old_index. then drop new_index, old_index
* feature engineering: number of counters per client. then drop client_id, counter_number
* turn into dummies: cat_features = ['district', 'client_catg', 'region', 'tarif_type', 'counter_statue', 'counter_code', 'reading_remarque', 'counter_coefficient', 'counter_type']
* export processed dataframe
* turn date columns into months and year columns


In [ ]:
# save current version of processed data for use later

#df_processed.to_csv('data/fraud_data_processed_V1.csv', index=False)


In [55]:
#store num-features
%store num_features

Stored 'num_features' (list)
